# Candlesticks Combined patterns analysis

In [1545]:
import yfinance as yf
import pandas as pd

#single Stock
df = yf.download('TSLA', 
                       start='2015-01-01', 
                       end='2022-07-16')

df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-11,756.309998,759.190002,700.880005,703.030029,703.030029,33080400
2022-07-12,710.539978,719.320007,685.109985,699.210022,699.210022,29310300
2022-07-13,676.500000,726.179993,675.099976,711.119995,711.119995,32651500
2022-07-14,704.690002,715.960022,688.000000,714.940002,714.940002,26185800
2022-07-15,720.000000,730.869995,710.669983,720.200012,720.200012,23165500


In [1546]:
#Check if NA values are in data
df.isna().sum()
df= df[df['Volume'] != 0]
df.reset_index(drop=True, inplace=True)


import pandas_ta as ta
#df.ta.indicators()
#help(ta.atr)
#df['ATR'] = df.ta.atr(length=10)

df.tail()


,Open,High,Low,Close,Adj Close,Volume
1892,756.309998,759.190002,700.880005,703.030029,703.030029,33080400
1893,710.539978,719.320007,685.109985,699.210022,699.210022,29310300
1894,676.500000,726.179993,675.099976,711.119995,711.119995,32651500
1895,704.690002,715.960022,688.000000,714.940002,714.940002,26185800
1896,720.000000,730.869995,710.669983,720.200012,720.200012,23165500


In [1560]:
# in total we have 1892 days (index)
length = len(df)

high = list(df['High'])
low = list(df['Low'])
close = list(df['Close'])
open = list(df['Open'])

bodydiff = [0] * length
highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.002:
        bodydiff[row]=0.002      

    bodydiffmin = 0.02
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        low[row]<low[row-1] and
        high[row]>high[row-1] and
        (open[row]<close[row-1]) and close[row]>1.02*open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
    
    
def isStar(l):
    bodydiffmin = 0.02
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.01:
        bodydiff[row]=0.01
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>2 and lowdiff[row]<0.4*highdiff[row] and bodydiff[row]>bodydiffmin): ## and open[row]>close[row]):
        return 1
    elif (ratio2[row]>2 and highdiff[row]<0.4*lowdiff[row] and bodydiff[row]>bodydiffmin):## open[row]<close[row]):
        return 2
    else:
        return 0


    

# Identifying the candle signal

In [1561]:
def Revsignal1():
    signal = [0] * length
    for row in range(1, length):
        if ((isEngulfing(row)==1 or isStar(row)==1) ): #and df.RSI[row]<30:
            signal[row] = 1
        elif((isEngulfing(row)==2 or isStar(row)==2) ): #and df.RSI[row]>60:
            signal[row] = 2
        else:
            signal[row] = 0
    return signal

df['signal'] = Revsignal1()
df.head(30)


,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result
0,44.574001,44.650002,42.652000,43.862000,43.862000,23822000,0,-0.712002,1.0,0
1,42.910000,43.299999,41.431999,42.018002,42.018002,26842500,0,-0.891998,2.0,0
2,42.012001,42.840000,40.841999,42.256001,42.256001,31309500,0,0.243999,1.0,0
3,42.669998,42.956001,41.956001,42.189999,42.189999,14842000,0,-0.480000,1.0,0
4,42.562000,42.759998,42.001999,42.124001,42.124001,17212500,0,-0.438000,1.0,0
5,41.784000,41.995998,40.992001,41.332001,41.332001,23341500,0,-0.452000,1.0,0
6,40.610001,40.894001,39.849998,40.442001,40.442001,29751500,0,-0.167999,2.0,0
7,40.664001,41.521999,40.181999,40.849998,40.849998,22386500,0,0.185997,2.0,0
8,37.166000,39.040001,37.000000,38.537998,38.537998,57759500,0,1.371998,1.0,0
9,38.897999,39.150002,38.000000,38.374001,38.374001,26082500,0,-0.523998,2.0,0


In [1562]:
#df.reset_index(drop=True, inplace=True)
df[df['signal']==2].count()

Open         146
High         146
Low          146
Close        146
Adj Close    146
Volume       146
signal       146
diff         146
realtrend    146
result       146
dtype: int64

In [1563]:
## Real stock market trend:
df['diff']=df['Close'] - df['Open']
df['realtrend'] = df['diff'].apply(lambda x: 2 if x > 0 else 1)

df['realtrend'] = df['realtrend'].shift(-1)
df.head(30)

,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result
0,44.574001,44.650002,42.652000,43.862000,43.862000,23822000,0,-0.712002,1.0,0
1,42.910000,43.299999,41.431999,42.018002,42.018002,26842500,0,-0.891998,2.0,0
2,42.012001,42.840000,40.841999,42.256001,42.256001,31309500,0,0.243999,1.0,0
3,42.669998,42.956001,41.956001,42.189999,42.189999,14842000,0,-0.480000,1.0,0
4,42.562000,42.759998,42.001999,42.124001,42.124001,17212500,0,-0.438000,1.0,0
5,41.784000,41.995998,40.992001,41.332001,41.332001,23341500,0,-0.452000,1.0,0
6,40.610001,40.894001,39.849998,40.442001,40.442001,29751500,0,-0.167999,2.0,0
7,40.664001,41.521999,40.181999,40.849998,40.849998,22386500,0,0.185997,2.0,0
8,37.166000,39.040001,37.000000,38.537998,38.537998,57759500,0,1.371998,1.0,0
9,38.897999,39.150002,38.000000,38.374001,38.374001,26082500,0,-0.523998,2.0,0


In [1564]:
import numpy as np

conditions = [(df['realtrend']==1) & (df['signal']==1), (df['realtrend']==2) & (df['signal']==2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

## trendID =2 means upward
## if trendID =1, then the accuracy rate increases to 0.7, but we don't short stocks so we only look at trendId = 2
trendId=2
print(df[df['result']==trendId].result.count()/df[df['signal']==trendId].signal.count())
#df[ (df['Trend']!=trendId) & (df['Trend']!=3) & (df['signal']==trendId) ] # false positives

print(df[df['result']==df['realtrend']].result.count()/df[df['signal']!=0].signal.count())
df[(df['signal']!=0)].tail(60)

0.6712328767123288
0.5842696629213483


,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result
1631,689.580017,693.809998,668.700012,671.869995,671.869995,32496700,1,-17.710022,2.0,0
1634,679.770020,692.809998,678.140015,679.700012,679.700012,18924900,1,-0.070007,1.0,1
1636,678.979980,700.000000,673.260010,678.900024,678.900024,27054500,1,-0.079956,1.0,1
1640,653.179993,658.909973,644.690002,656.950012,656.950012,18140500,2,3.770020,2.0,2
1652,663.400024,666.500000,627.239990,644.780029,644.780029,32813300,1,-18.619995,1.0,1
1658,711.000000,724.900024,708.929993,710.919983,710.919983,17002600,1,-0.080017,1.0,1
1662,713.989990,716.590027,701.880005,709.989990,709.989990,13432300,1,-4.000000,1.0,1
1667,672.659973,674.580017,648.840027,665.710022,665.710022,23721300,2,-6.949951,2.0,2
1679,734.500000,740.969971,730.539978,732.390015,732.390015,12777300,1,-2.109985,2.0,0
1680,732.250000,734.000000,724.200012,733.570007,733.570007,15246100,2,1.320007,2.0,2


In [1552]:
dfpl = df[1540:1600]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [1523]:
# buy at open price and sell at close price

In [1528]:
## Take past year 365 days dataframe
trading_day_pastyear = df[1000:]

##only take the day which has signal 2: upward trend
##trading_day = past_year[past_year['signal']==2]

##Assume I have $50,000 initial budget to invest in TSLA
## we can buy (50000/open price) shares TSLA and sell all of them at close price
length2 = len(trading_day_pastyear)
trading_day_pastyear['position']= 0 * length2
trading_day_pastyear['profit'] = 0 * length2

/var/folders/dg/zr94h80j0kn6rb0ldv3kx8cr0000gn/T/ipykernel_83213/2827099458.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dg/zr94h80j0kn6rb0ldv3kx8cr0000gn/T/ipykernel_83213/2827099458.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1529]:
## take a look at last 30 tradings from TSLA

trading_day_pastyear.head(31)


,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result,position,profit
1000,39.215000,39.540001,37.407501,37.682499,36.481178,382978400,0,-1.532501,1.0,0,0,0
1001,37.037498,37.887501,36.647499,36.707500,35.537262,148676800,1,-0.329998,2.0,0,0,0
1002,37.075001,39.307499,36.680000,39.292500,38.039845,234330000,0,2.217499,2.0,0,0,0
1003,38.959999,39.192501,37.517502,39.037498,37.792976,212468400,2,0.077499,1.0,0,0,0
1004,39.375000,39.630001,38.637501,39.057499,37.812336,169165600,0,-0.317501,1.0,0,0,0
1005,39.632500,39.840000,39.119999,39.435001,38.177799,140014000,0,-0.197498,2.0,0,0,0
1006,38.722500,39.712502,38.557499,39.480000,38.221367,148158800,0,0.757500,1.0,0,0,0
1007,35.994999,36.430000,35.500000,35.547501,34.414230,365248800,0,-0.447498,2.0,0,0,0
1008,36.132500,37.137501,35.950001,37.064999,35.883366,234428400,0,0.932499,1.0,0,0,0
1009,37.174999,37.207500,36.474998,36.982498,35.803486,219111200,2,-0.192501,2.0,2,0,0


In [1530]:
trading_day_pastyear.info()
## in total 365 times trading for the whole year for TSLA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 1000 to 1896
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       897 non-null    float64
 1   High       897 non-null    float64
 2   Low        897 non-null    float64
 3   Close      897 non-null    float64
 4   Adj Close  897 non-null    float64
 5   Volume     897 non-null    int64  
 6   signal     897 non-null    int64  
 7   diff       897 non-null    float64
 8   realtrend  896 non-null    float64
 9   result     897 non-null    int64  
 10  position   897 non-null    int64  
 11  profit     897 non-null    int64  
dtypes: float64(7), int64(5)
memory usage: 84.2 KB


In [1531]:
# profit at 12, open at 0, diff at 9, position at 11

investment = 50000
for i in range (len(trading_day_pastyear)):
    if trading_day_pastyear.iloc[i,6] == 2:
        trading_day_pastyear.iloc[i+1, 11] = investment/trading_day_pastyear.iloc[i+1,0] * trading_day_pastyear.iloc[i+1, 7]
        investment += trading_day_pastyear.iloc[i+1,11]
        trading_day_pastyear.iloc[i+1,10] = investment
        
investment-50000




/var/folders/dg/zr94h80j0kn6rb0ldv3kx8cr0000gn/T/ipykernel_83213/1225513032.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dg/zr94h80j0kn6rb0ldv3kx8cr0000gn/T/ipykernel_83213/1225513032.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



111.82335893763957

In [1532]:
trading_day_pastyear.head(50)

,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result,position,profit
1000,39.215000,39.540001,37.407501,37.682499,36.481178,382978400,0,-1.532501,1.0,0,0.000000,0.000000
1001,37.037498,37.887501,36.647499,36.707500,35.537262,148676800,1,-0.329998,2.0,0,0.000000,0.000000
1002,37.075001,39.307499,36.680000,39.292500,38.039845,234330000,0,2.217499,2.0,0,0.000000,0.000000
1003,38.959999,39.192501,37.517502,39.037498,37.792976,212468400,2,0.077499,1.0,0,0.000000,0.000000
1004,39.375000,39.630001,38.637501,39.057499,37.812336,169165600,0,-0.317501,1.0,0,49596.824040,-403.175960
1005,39.632500,39.840000,39.119999,39.435001,38.177799,140014000,0,-0.197498,2.0,0,0.000000,0.000000
1006,38.722500,39.712502,38.557499,39.480000,38.221367,148158800,0,0.757500,1.0,0,0.000000,0.000000
1007,35.994999,36.430000,35.500000,35.547501,34.414230,365248800,0,-0.447498,2.0,0,0.000000,0.000000
1008,36.132500,37.137501,35.950001,37.064999,35.883366,234428400,0,0.932499,1.0,0,0.000000,0.000000
1009,37.174999,37.207500,36.474998,36.982498,35.803486,219111200,2,-0.192501,2.0,2,0.000000,0.000000


In [1533]:
trading_day_pastyear[trading_day_pastyear['profit']!=0]

,Open,High,Low,Close,Adj Close,Volume,signal,diff,realtrend,result,position,profit
1004,39.375000,39.630001,38.637501,39.057499,37.812336,169165600,0,-0.317501,1.0,0,49596.824040,-403.175960
1010,37.389999,37.955002,37.130001,37.687500,36.486012,164101200,0,0.297501,2.0,0,49991.450563,394.626522
1024,39.062500,39.532501,38.527500,38.669998,37.437195,166348800,0,-0.392502,2.0,0,49489.134124,-502.316439
1047,43.922501,44.437500,43.492500,43.962502,42.743500,109744800,0,0.040001,1.0,0,49534.204661,45.070537
1056,46.212502,46.832500,45.935001,46.529999,45.239811,156171600,0,0.317497,2.0,0,49874.523263,340.318602
...,...,...,...,...,...,...,...,...,...,...,...,...
1846,158.149994,160.710007,156.320007,159.479996,159.246017,88966500,0,1.330002,2.0,0,48535.570358,404.767991
1857,146.850006,147.360001,139.899994,140.820007,140.820007,109742900,0,-6.029999,1.0,0,46542.588282,-1992.982076
1860,137.789993,143.259995,137.649994,143.110001,143.110001,117726300,0,5.320007,1.0,0,48339.575891,1796.987609
1862,138.429993,141.789993,138.339996,140.520004,140.520004,92482700,0,2.090012,2.0,0,49069.405259,729.829369
